# BYOC TensorFlow(TensorFlow 커스텀 컨테이너 사용하기)

***Note: 본 핸즈온은 SageMaker SDK V2에 대응하도록 코드를 수정했습니다. 만약 SDK V1용 코드가 필요하다면 아래 링크를 참조해 주십시오.***

소스 : https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/tensorflow_bring_your_own/tensorflow_bring_your_own.ipynb

Amazon SageMaker에서는 여러분의 알고리즘을 SageMaker 환경에서 학습하고 배포할 수 있도록 패키징할 수 있습니다. 본 노트북은 Tensorflow를 SageMaker환경에서 학습, 배포할 수 있도록 커스텀 도커 컨테이너로 빌드하는 방법을 알아봅니다.

알고리즘을 컨테이너로 패키징함으로써, 프로그래밍 언어나 프레임워크, 의존관계 등과 무관하게 여러분의 코드를 SageMaker 환경에서 구동할 수 있게 됩니다. 

설명없이 코드로 바로 시작하시려면 __도커파일 (코드 시작)__ 섹션으로 바로 이동하십시오.  

## 언제 커스텀 알고리즘 컨테이너를 사용해야 하나요?

여러분의 코드를 SageMaker에서 구동하기 위하여 반드시 컨테이너를 직접 만들어야 하는 것은 아닙니다. 만약 여러분이 사용하는 프레임웤이 Apach MXNet이나 Tensorflow 등일 경우 SageMaker는 해당 프레임워크를 직접 지원하기 때문에 여러분은 알고리즘을 구현하는 파이썬 코드만 제공하고 이를 프레임워크 SDK의 entry point에 전달하여 활용할 수 있습니다. SageMaker에서 지원되는 프레임워크는 정기적으로 추가되고 있고, SageMaker에서 지원하는 범용 머신러닝 환경 리스트를 통해 여러분이 작성하시는 알고리즘의 환경이 SageMaker에서 지원되는 지를 확인할 수 있습니다. 

단, 여러분의 환경이나 프레임워크를 지원하는 SDK가 있다고 하더라도 커스텀 컨테이너를 직접 빌드하는 것이 보다 효과적일 경우도 있습니다. 여러분의 알고리즘 코드가 매우 복잡하거나 추가로 다른 프레임웤를 필요로 하는 경우에는 컨테이너를 직접 빌드하는 것이 적절한 선택일 수도 있습니다. 이런 경우에 해당하는 몇가지 사례는 다음과 같습니다. 

1. 프레임워크의 특정 버전이 지원되지 않는 경우
2. 환경에 의존라이브러리들을 추가로 설치하고 설정하는 경우
3. 기본 환경에서 제공되지 않는 학습/배포 솔루션을 사용하는 경우

커스텀 컨테이너를 이용하면 SageMaker에서 사전 제공하지 않는 환경일 경우에도 SageMaker기반으로 동작하도록 할 수 있습니다. 본 예제에서 그 과정을 살펴보겠습니다. 

## 권한 설정

본 노트북은 `SageMakerFullAccess`권한에 추가로 Amazon ECR에 접근하기 위한 권한이 필요합니다. 권한을 추가하는 가장 간단한 방법은 관리형 정책인 `AmazonEC2ContainerRegistryFullAccess`를 노트북 인스턴스가 사용중인 역할(role)에 추가하는 것입니다. 이를 위해 노트북 인스턴스를 재시작할 필요는 없으며 수정 즉시 새로운 권한이 할당 될 것입니다. 


## 샘플 시나리오

본 예제에서는 CIFAR-10 데이터셋을 학습할 수 있는 커스텀 Tensorflow 컨테이너를 패키징하고 추론 환경은 Tensorflow Serving을 사용합니다. 하지만 본 예제를 일부 수정하면 Tensorflow Serving이 아닌 다른 추론 환경에도 응용할 수 있을 것입니다. 

[CIFAR-10]: http://www.cs.toronto.edu/~kriz/cifar.html

본 예제에서는 학습과 추론 실행을 단일 이미지로 사용할 것입니다. 하나의 이미지로 학습과 추론을 실행하는 경우 하나의 이미지만 관리하면 되므로 관리절차가 단순해 집니다. 하지만 실제 요구사항에 따라 이 두 환경의 이미지가 분리될 수도 있을 것입니다. 이 경우 Dockerfile을 분리하고 두 개의 이미지를 운영하게 됩니다. 이미지의 분리와 통합은 실제 환경 요구사항과 개발 및 관리 편의성을 고려하여 결정합니다. 

그리고 SageMaker에서 학습과 추론을 모두 실행하지 않고 이 중 한가지 방식만 이용할 경우에는 해당 기능의 이미지만 빌드하면 됩니다.

본 예제는 컨테이너를 __빌드__하는 부분과 __활용__하는 부분의 두 파트로 나누어져 있습니다. 


# Part 1: 알고리즘을 SageMaker환경으로 패키징하고 업로드하기 

### 도커 개요

만약 여러분이 이미 도커 환경이 친숙하다면 이 단계는 건너뛰고 다음 섹션으로 이동하십시오.

많은 데이터 사이언티스트들에게 도커 컨테이너는 새로운 기술입니다. 하지만 도커는 여러분의 소프트웨어를 패키징하여 배포하는 매우 간편한 방법이며 그리 어렵지 않습니다. 

도커는 임의의 코드를 스스로 실행환경을 포함하는 __이미지__로 패키징하는 간단한 방법을 제공합니다. 이미지가 생성되고 나면, 도커 환경에서 이 이미지를 __컨테이너__로 실행하게 됩니다. 컨테이너를 실행하는 것은 컨테이너가 프로그램이 실행될 실행환경을 스스로 생성한다는 것을 제외하면 일반적으로 컴퓨터 머신에서 프로그램을 실행하는 것과 다르지 않습니다. 컨테이너는 호스트 환경에서 다른 컴퓨팅 환경과 격리됩니다. 이것은 이 컨테이너가 어디에 실행되는지에 관계없이 독자적인 실행방식을 구성할 수 있게 만들어 줍니다.

도커는 다음과 같은 이유로 conda나 virtualenv와 같은 환경관리에 비해 매우 강력합니다. 
- 도커 환경은 프로그래밍 언에에 독립적입니다.
- 초기 실행명령과 환경변수까지 전체 운영환경을 포괄합니다.

도커 컨테이너는 가상머신과 유사하지만 보다 경령화되어 있습니다. 예를 들어, 컨테이너에서 실행되는 프로그램은 1초 이내에 시작될 수 있고 물리 또는 가상 서버 인스턴스에서 동시에 실행될 수 있습니다. 

도커는 `Dockerfile`이라는 간단한 파일을 사용하여 이미지가 구성되는 방식을 정의합니다. 그 사례는 아래에 제공됩니다. 여러분은 여러분이 만들거나 또는 다른 사람이 만든 도커 이미지를 기반으로 새로운 도커 이미지를 만들 수 있습니다. 이런 방식은 이미지의 생성을 매우 단순화시켜줍니다.

도커는 이런 유연함과 함께 잘 정리된 컨테이너 설정방식을 제공하여 프로그래밍과 개발 커뮤니테에서 열광적인 환영을 받고 있으며, 최근 [Amazon ECS]나 [Amazon EKS]와 같은 많은 서비스가 이 기술을 기반으로 구현되고 제공되고 있습니다. 

Amazon SageMaker 또한 사용자가 임의의 알고리즘을 학습하고 배포할 때 이 도커를 이용합니다. 

Amazon SageMaker 에서 도커 컨테이너는 학습을 위한 방식과, 학습과는 조금 다른 추론 호스팅의 방식으로 호출(invoke) 됩니다. 다음 섹션에서 SageMaker 환경에서 컨테이너를 어떻게 빌드하는지에 대해 설명합니다. 

도커와 관련한 보다 자세한 내용은 아래 링크를 참고하십시오. 

* [Docker home page](http://www.docker.com)
* [Getting started with Docker](https://docs.docker.com/get-started/)
* [Dockerfile reference](https://docs.docker.com/engine/reference/builder/)
* [`docker run` reference](https://docs.docker.com/engine/reference/run/)

[Amazon ECS]: https://aws.amazon.com/ecs/
[Amazon EKS]: https://aws.amazon.com/eks/

### Amazon SageMaker 에서 도커 컨테이너 실행 방식 

SageMaker에서 학습과 호스팅에 동일한 이미지를 사용할 수 있으며, 컨테이너를 실행시 `train` 또는 `serve`라는 매개변수(argument)를 이용하여 컨테이너를 실행합니다. 컨테이너가 이 매개변수를 처리하는 방식은 컨테이너 구성에 따라 달라집니다.

* 본 샘플에서는 도커파일(Dokerfile)에서 `ENTRYPOINT`를 사용하지 않습니다. 대신 학습시점에는 [`train`](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html) 명령을 서빙(호스팅)시점에는 [`serve`](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html) 명령을 이용하여 도커를 실행합니다.
* 만약 도커파일에서 `ENTRYPOINT`를 이용하여 프로그램을 명시하게 되면 도커 실행시 해당프로그램이 실행되며 `train` 또는 `serve`가 첫번째 매개변수로 전달될 것입니다. 프로그램은 이 매개변수에 따라 무슨 동작을 해야할 지를 판단할 수 있습니다. 
* 만약 학습과 호스팅 컨테이너를 분리하는 경우 (또는 둘 장 하나의 방식만 활용하는 경우) 도커파일에 `ENTRYPOINT`에 프로그램을 정의할 수 있고, 이 때 전달되는 첫번째 매개변수는 무시하거나 또는 검증용으로 사용할 수 있습니다. 


#### 학습(training)용으로 커스텀 컨테이너 이용하기 

SageMaker에서 학습이 실행될 때 여러분의 `train` 스크립트가 실행됩니다. 일반적인 파이썬 프로그램이 실행되는 것과 마찬가지 방식이며, `/opt/ml` 디렉토리의 몇가지 파일들이 활용될 수 있습니다.

    /opt/ml
    |-- input
    |   |-- config
    |   |   |-- hyperparameters.json
    |   |   `-- resourceConfig.json
    |   `-- data
    |       `-- <channel_name>
    |           `-- <input data>
    |-- model
    |   `-- <model files>
    `-- output
        `-- failure

##### 데이터 입력

* `/opt/ml/input/config` - 프로그램이 실행되는 방식을 관리할 수 있는 정보가 포함됩니다. `hyperparameters.json` 은 하이퍼파라미터 이름과 값을 가지는 JSON 형식의 딕셔너리입니다. 값은 string 형태로 전달되며 필요시 타입을 변환하여 사용합니다. `resourceConfig.json` 은 분산학습에서 네트워크 레이아웃을 알려주는 JSON형식의 파일입니다. 

* `/opt/ml/input/data/<channel_name>/` - (파일 모드일 경우) 채널에 대한 입력데이터가 저장됩니다. 채널은 CreateTrainingJob(또는 fit)을 호출할 때 생성되며 알고리즘 코드에서 동일하게 사용해야 합니다. 알고리즘에서 사용할 파일들이 S3로부터 지정한 채널로 복사되며 S3 키의 트리 구조를 유지합니다. 

* `/opt/ml/input/data/<channel_name>_<epoch_number>` (파이프 모드일 경우) 실행되는 epoch을 위한 파이프입니다. epoch은 0에서부터 이 채널을 읽을때마다 증가합니다. epoch숫자에 제한은 없으며 다음 epoch을 실행하기 전에 pipe를 close 해야 합니다. 


##### 학습결과 출력

* `/opt/ml/model/` - 알고리즘이 생성하는 모델이 저장되는 디렉토리입니다. 모델의 형식은 여러분이 지정하는 방식에 따라 달라지며, 단일 파일일 수도 있고 트리구조를 가지는 디렉토리 전체일 수도 있습니다. 모델 파일은 `DescribeTrainingJob` 호출 결과로 리턴되는 지정된 S3 위치에 사용가능하도록 export 됩니다. 

* `/opt/ml/output` - 작업이 실패할 때 `failure` 파일을 저장하는 디렉토리입니다. 파일의 내용은 `DescribeTrainingJob`호출시 `FailureReason` 필드의 값으로 리턴됩니다. 작업이 성공적으로 종료될 경우 이 파일은 필요가 없으므로 무시됩니다. 


#### 호스팅(추론)용으로 컨테이너 이용하기

추론 호스팅은 HTTP를 통해 들어오는 추론 요청(request)에 응답해야 하므로 학습과는 다른 모델을 가집니다. 본 사례에서는 [TensorFlow Serving](https://www.tensorflow.org/serving/)을 사용하지만 이는 다른 환경으로 커스터마이징될 수도 있습니다. [Python serving stack within the scikit learn example](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb)과 같은 사례를 참고하십시오. 

Amazon SageMaker 추론 컨테이너 내부에서는 다음 두가지 URL을 사용합니다. 

* `/ping` - 인프라로부터 `GET` 요청을 받아서 처리합니다. 컨테이너가 정상적으로 동작하고 요청을 받을 수 있는 경우 200을 리턴합니다.
* `/invocations` - 추론 클라이언트로부터 `POST`요청을 받아 처리합니다. 요청과 응답의 형식은 알고리즘에 따라 달라집니다. 클라이언트가 `ContentType`과 `Accept` 헤더를 지정한 경우 함께 전달됩니다.

추론 컨테이너의 모델 파일 위치는 학습에서 생성한 모델을 저장할 때 사용한 위치와 동일합니다. 

    /opt/ml
    `-- model
        `-- <model files>



### The parts of the sample container

본 예제의 `container` 디렉토리에 샘플 알고리즘을 SageMaker환경으로 패키징하는 모든 파일들이 저장되어 있습니다.

    .
    |-- Dockerfile
    |-- build_and_push.sh
    `-- cifar10
        |-- cifar10.py
        |-- resnet_model.py
        |-- nginx.conf
        |-- serve
        `-- train

이들 파일을 차례로 살펴보면:

* __`Dockerfile`__ - 도커 컨테이너 이미지를 빌드하는 방법이 정의됩니다. 다음 섹션에서 다시 설명합니다. 
* __`build_and_push.sh`__ - Dockerfile을 이용하여 컨테이너 이미지를 빌드하고 ECR로 push하는 스크립트입니다. 본 노트북에서 이 쉘을 직접 실행할 것입니다. 이후 여러분의 알고리즘에 적용할 경우에도 이 파일을 그대로 사용할 수 있습니다. 
* __`cifar10`__ - 컨테이너 내부로 복제될 파일들이 저장되어 있습니다. 

본 예제에서 우리는 5개의 파일을 컨테이너로 복제할 것입니다. 실제로는 여러분의 유즈케이스에 따라 이정도의 파일이 충분할 수도 있고 또는 더 많은 파일이 필요할 수도 있습니다. 하지만 이 5개의 파일이 커스텀 파이썬 컨테이너의 표준 구조가 됩니다. 물론 예제코드와 다른 도구(toolkit)를 사용할 경우에는 다른 구조가 될 수도 있습니다. 본 예제는 Tensorflow serving과 nginx를 사용하지만 다른 프로그래밍 언어나 프레임워크를 사용하는 경우 다른 구성을 가질 것입니다. 

본 예제에서 컨테이너에 복제될 파일은 다음과 같습니다. (`cifar10`디렉토리 내부의 파일들)

* __`cifar10.py`__ - 알고리즘의 실행을 구현하는 프로그램 코드입니다. 
* __`resnet_model.py`__ - Resnet 모델을 정의하는 코드입니다. (`cifar10.py`에서 사용합니다.)
* __`nginx.conf`__ - nginx front-end를 구성하는 설정 파일입니다. 일반적으로 제공되는 파일을 그대로 사용가능합니다.
* __`serve`__ - 컨테이너가 추론 호스팅을 할 때 실행되는 프로그램입니다. nginx를 실행하고 Tensorflow serving에 모델을 로드합니다. 
* __`train`__ - 컨테이너가 학습을 진행할 때 실행되는 프로그램입니다. 본 예제에서는 /opt/ml/input/config/hyperparameters.json로 입력된 하이퍼파라미터를 이용하여 `cifar10.py` 를 호출(invoke)할 것입니다. 이런 구조를 통해 학습알고리즘과 컨테이너 실행코드를 독립적으로 분리하여 관리합니다.

요약하면, 이후 여러분의 실제 애플리케이션에 여러분의 알고리즘 실행코드를 적용할 때에는 해당 코드와 함께 `train`과 `serve` 부분을 변경하게 됩니다. 


### 도커파일 (코드 시작)

Dockerfile은 빌드할 이미지를 정의합니다. 실행할 시스템의 운영체제와 환경을 완전히 정의하는 것으로 생각해도 좋습니다. 하지만 실제로 도커 컨테이너의 실행은 운영체제를 모두 준비하는 것에 비해 매우 경량화되어 있으며 기초 동작은 호스트머신의 Linux를 활용합니다. 

파이선 데이터사이언스 스택을 준비하기 위해 Tensorflow docker 이미지로 부터 시작하하여 기본 도구와 Tensorflow serving을 설치하겠습니다. 그리고 이 환경에서 실행될 사용자 알고리즘 코드를 추가하겠습니다. 

아래 코드를 이용하여 Dockerfile을 살펴봅니다.

In [1]:
!cat container/Dockerfile

# Copyright 2017-2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License"). You
# may not use this file except in compliance with the License. A copy of
# the License is located at
#
#     http://aws.amazon.com/apache2.0/
#
# or in the "license" file accompanying this file. This file is
# distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF
# ANY KIND, either express or implied. See the License for the specific
# language governing permissions and limitations under the License.

# For more information on creating a Dockerfile
# https://docs.docker.com/compose/gettingstarted/#step-2-create-a-dockerfile
FROM tensorflow/tensorflow:1.8.0-py3

RUN apt-get update && apt-get install -y --no-install-recommends nginx curl

# Download TensorFlow Serving
# https://www.tensorflow.org/serving/setup#installing_the_modelserver
RUN echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable te

### 컨테이너 빌드 및 등록

다음 쉘스크립트는 `docker build` 명령을 이용하여 컨테이너를 빌드하고 `docker push`명령을 이용하여 ECR에 빌드한 이미지를 push하는 방법을 보여줍니다. 해당 코드는 `container/build-and-push.sh`의 내용과 동일하며 `sagemaker-tf-cifar10-example`라는 이름으로 이미지를 빌드하고 push 하고자 할 때 `build-and-push.sh sagemaker-tf-cifar10-example`와 같은 형식으로 실행하면 됩니다.

아래 코드는 여러분의 어카운트의 디폴트 리전 (또는 SageMaker 노트북 인스턴스를 사용중인 경우 노트북이 생성된 리전)에서 ECR 레포지토리를 찾고, 만약 레포지토리가 없다면 이를 생성할 것입니다. 


In [2]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-tf-cifar10-example

cd container

chmod +x cifar10/train
chmod +x cifar10/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
# region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/8 : FROM tensorflow/tensorflow:1.8.0-py3
1.8.0-py3: Pulling from tensorflow/tensorflow
297061f60c36: Pulling fs layer
e9ccef17b516: Pulling fs layer
dbc33716854d: Pulling fs layer
8fe36b178d25: Pulling fs layer
686596545a94: Pulling fs layer
11251ff8ca17: Pulling fs layer
632ad5221740: Pulling fs layer
c61bc0eaf8d9: Pulling fs layer
f5e02d7ae67a: Pulling fs layer
f29a5775b72a: Pulling fs layer
c9bf16bc9d22: Pulling fs layer
d4423e297b68: Pulling fs layer
418a4217796d: Pulling fs layer
02b18d984060: Pulling fs layer
686596545a94: Waiting
c9bf16bc9d22: Waiting
c61bc0eaf8d9: Waiting
11251ff8ca17: Waiting
d4423e297b68: Waiting
418a4217796d: Waiting
f29a5775b72a: Waiting
632ad5221740: Waiting
f5e02d7ae67a: Waiting
02b18d984060: Waiting
8fe36b178d25: Waiting
e9ccef17b516: Verifying Checksum
e9ccef17b516: Download complete
dbc33716854d: Verifying Checksum
dbc33716854d: Download complete
8fe36b178d25: Verifying Checksum
8fe36b178d25: Download complete
686596545a94: Veri

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



## 로컬 머신 (또는 SageMaker 노트북 인스턴스)에서 알고리즘 테스트 

알고리즘을 처음 패키징할 때에는 여러분의 코드가 잘 동작하는지 테스트가 필요할 것입니다. [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk)를 이용하여 SageMaker환경 또는 로컬환경에서 이를 확인할 수 있습니다. SageMaker Python SDK와 관련한 더 많은 예제는 [Amazon SageMaker Examples](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/sagemaker-python-sdk)를 참고합니다.

이제 테스트를 위해 데이터셋을 준비합니다. 

## CIFAR-10 데이터셋 다운로드

본 예제에서 Tensorflow 학습 실행시 레코드 기반의 이진 포맷인 [TFRecords](https://www.tensorflow.org/guide/datasets) 형식을 이용할 것입니다.
다음 코드는 [official TensorFlow CIFAR-10 example](https://github.com/tensorflow/models/tree/master/tutorials/image/cifar10_estimator) 공식 예제이며 CIFAR-10 데이터셋을 다운로드하고 TFRecord로 변환합니다.


In [5]:
! python utils/generate_cifar10_tfrecords.py --data-dir=/tmp/cifar-10-data


Download from https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz and extract.
FloatProgress(value=0.0)
>> Downloading cifar-10-python.tar.gz 
Successfully downloaded cifar-10-python.tar.gz 170498071 bytes.
Generating /tmp/cifar-10-data/train.tfrecords


Generating /tmp/cifar-10-data/validation.tfrecords
Generating /tmp/cifar-10-data/eval.tfrecords
Removing original files.
Done!


In [6]:
# There should be three tfrecords. (eval, train, validation)
! ls /tmp/cifar-10-data

eval.tfrecords	train.tfrecords  validation.tfrecords


## SageMaker Python SDK 로컬 학습(trianing)

학습을 진행하기 위해 Estimator를 선언하며 다음을 파라미터로 구성합니다.
1. IAM role - AWS 실행 역할(role)
2. train_instance_count - 학습에 사용할 인스턴스 개수 
3. train_instance_type - 학습에 사용할 인스턴스 타입. 로컬에서 실행하려면 `local`로 입력함
4. image_name - (앞 단계에서 생성한) 커스텀 TensorFlow 도커 이미지
5. hyperparameters - 학습에 사용할 하이퍼파리미터들

IAM 역할을 구성해보겠습니다. SageMaker Python SDK의 helper function을 이용합니다. 이 함수는 SageMaker 노트북 인스턴스로부터 메터정보를 얻기 때문에, SageMaker 노트북 환경이 아닌 경우 에러가 발생할 것입니다. SageMaker 노트북 환경이 아닌경우 적절한 권한을 가지는 역할을 지정합니다. 


In [7]:
from sagemaker import get_execution_role

role = get_execution_role()

## Fit, Deploy, Predict

Estimator의 나머지 내용을 구성하고 `file://`로 시작하는 로컬 CIFAR10 데이터셋 경로를 이용하여 `fit()`을 호출합니다. 이제 SageMaker는 Tensorflow 컨테이너의 'train' 을 실행하면서 하이퍼파리미터와 다른 메타데이터들을 /opt/ml/input/config 경로로 전달할 것입니다.

학습 작업이 성공적으로 종료되면, 알고리즘은 /opt/ml/model 디렉토리에 학습된 모델을 저장합니다. 이 모델은 예측에 이용될 것입니다.

학습이후 instance_count와 instance_type을 각각 1과 `local`로 지정하고 `deploy()`를 호출합니다. SageMaker는 Tensorflow 컨테이너의 'serve'를 실행할 것입니다. serve 프로세스는 이제 Tensorflow serving을 통해 예측 요청을 처리하는 셋업을 컨테이너에 실행합니다. 'deploy'의 결과르 predictor라 리턴되며 이는 학습된 모델을 이용하여 추론을 하는 데 사용됩니다. 

예측이 끝난 후에는 엔드포인트를 삭제할 수 있습니다.

이처럼 보다 빠른 피드백을 받으면서 디버깅을 할 수 있도록, 먼저 로컬환경에서 알고리즘을 테스트해볼 것을 추천합니다. 


In [8]:
# Lets set up our SageMaker notebook instance for local mode.
!/bin/bash ./utils/setup.sh

The user has root access.
nvidia-docker2 already installed. We are good to go!
SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


### Training(학습)

In [25]:
from sagemaker.estimator import Estimator

hyperparameters = {'train-steps': 100}

instance_type = 'local'

estimator = Estimator(role=role,
                      instance_count=1,
                      instance_type=instance_type,
                      image_uri='sagemaker-tf-cifar10-example:latest',
                      hyperparameters=hyperparameters)

estimator.fit('file:///tmp/cifar-10-data')

Creating 869wbx0a5m-algo-1-t8grk ... 
Creating 869wbx0a5m-algo-1-t8grk ... done
Attaching to 869wbx0a5m-algo-1-t8grk
869wbx0a5m-algo-1-t8grk | Training complete.
869wbx0a5m-algo-1-t8grk exited with code 0
Aborting on container exit...
===== Job Complete =====


### Deployment(추론)

2020년 10월부터 SageMaker Python SDK가 v1에서 v2로 전면 변경되면서, 요청(request)과 응답(response)의 용법이 크게 변경되었습니다. 자세한 변경점은 아래 링크를 참조해 주십시오.
- https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/upgrade_from_legacy.html

In [26]:
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer
predictor = estimator.deploy(1, instance_type, serializer=JSONSerializer(), deserializer=JSONDeserializer())

Attaching to h14g8h4f7a-algo-1-3i3vh
h14g8h4f7a-algo-1-3i3vh | Starting TensorFlow Serving.
h14g8h4f7a-algo-1-3i3vh | 2021-03-20 08:26:17.622246: I tensorflow_serving/model_servers/server.cc:88] Building single TensorFlow model file config:  model_name: cifar10_model model_base_path: /opt/ml/model/export/Servo
h14g8h4f7a-algo-1-3i3vh | 2021-03-20 08:26:17.622441: I tensorflow_serving/model_servers/server_core.cc:464] Adding/updating models.
h14g8h4f7a-algo-1-3i3vh | 2021-03-20 08:26:17.622467: I tensorflow_serving/model_servers/server_core.cc:587]  (Re-)adding model: cifar10_model
h14g8h4f7a-algo-1-3i3vh | 2021-03-20 08:26:17.722810: I tensorflow_serving/core/basic_manager.cc:740] Successfully reserved resources to load servable {name: cifar10_model version: 1616228549}
h14g8h4f7a-algo-1-3i3vh | 2021-03-20 08:26:17.722844: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: cifar10_model version: 1616228549}
h14g8h4f7a-algo-1-3i3vh | 2021-03-20 08

## Python SDK로 추론 실행하기

예측을 실행하기 위해 OpenCV를 이용하여 이미지를 json 형식으로 변환하여 사용하겠습니다. OpenCV를 설치합니다. 

JSON 응답 결과는 10개의 클래스 중 해당 사진이 속할 클래스의 확률을 리턴할 것입니다. 클래스는 다음 [CIFAR-10 website](https://www.cs.toronto.edu/~kriz/cifar.html) 링크를 참조합니다. 학습을 오래 실행하지는 않았으므로 아직 결과의 정확도는 떨어질 것입니다.


In [27]:
! pip install opencv-python

In [28]:
import cv2
import numpy
image = cv2.imread("data/cat.png", 1)

# resize, as our model is expecting images in 32x32.
image = cv2.resize(image, (32, 32))
data = {'instances': numpy.asarray(image).astype(float).tolist()}

# For more information on the predictor class.
# https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/predictor.py
predictor.predict(data)

{'predictions': [{'probabilities': [0.00115666038,
    0.905890524,
    0.00990418904,
    0.00254561636,
    0.00637906278,
    0.0192576591,
    0.0019765608,
    0.00477025565,
    0.0402701311,
    0.00784943439],
   'classes': 1}]}

h14g8h4f7a-algo-1-3i3vh | 172.18.0.1 - - [20/Mar/2021:08:26:31 +0000] "POST /invocations HTTP/1.1" 200 262 "-" "python-urllib3/1.26.3"


In [29]:
!docker ps -a

CONTAINER ID        IMAGE                                 COMMAND             CREATED             STATUS                     PORTS                                        NAMES
2667796ab8e0        sagemaker-tf-cifar10-example:latest   "serve"             17 seconds ago      Up 15 seconds              6006/tcp, 8888/tcp, 0.0.0.0:8080->8080/tcp   h14g8h4f7a-algo-1-3i3vh
02823d3c317f        sagemaker-tf-cifar10-example:latest   "train"             4 minutes ago       Exited (0) 4 minutes ago                                                869wbx0a5m-algo-1-t8grk


In [30]:
predictor.delete_endpoint()

Gracefully stopping... (press Ctrl+C again to force)


# Part 2: Amazon SageMaker에서 커스텀 알고리즘으로 학습과 추론 실행

컨테이너 패키징이 완료되면 이제 SageMaker에서도 학습과 추론을 실행할 수 있습니다. 앞서 만든 알고리즘 컨테이너를 그대로 사용합니다. 


## 환경 셋업

SageMaker에서 사용할 S3 버킷을 설정합니다. 


In [32]:
import string, random
string_pool = string.ascii_lowercase + string.ascii_uppercase + string.digits
result = "" 
for i in range(8) : 
    result += random.choice(string_pool)

In [33]:
# S3 prefix
prefix = f'DEMO-tensorflow-cifar10-{result}'
print(prefix)

DEMO-tensorflow-cifar10-OHZGFSkc


## 세션 생성

세션은 SageMaker 환경에 대한 접속 파리미터를 기억합니다. 이후 SageMaker 동작에 이 세션을 사용할 것입니다. 


In [34]:
import sagemaker as sage
sess = sage.Session()

## 학습용 데이터 업로드

SageMaker Python SDK에서 제공되는 도구를 이용하여 데이터를 디폴트 버킷에 업로드합니다.

In [35]:
WORK_DIRECTORY = '/tmp/cifar-10-data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

## SageMaker에서 학습(training)

SageMaker에서의 학습은 Python SDK를 통해 실행할 수 있으며 local 환경에서 실행했던 방식과 거의 동일합니다. 
- 이전 단계에서 `local`로 지정했던 train_instance_type 을 [supported EC2 instance types](https://aws.amazon.com/sagemaker/pricing/instance-types/) 리스트중 하나로 변경하여 지정합니다. 
- 추가로, 이전 단계에서 ECR에 push했던 이미지 URL을 지정합니다. 
- 마지막으로, S3로 업로드한 학습 데이터셋의 S3 URL을 지정하여 `fit()` 을 호출합니다.

다음 코드는 이전 단계에서 push한 ECR 이미지 URL을 먼저 가져오고 있습니다.


In [37]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'sagemaker-tf-cifar10-example'

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

143656149352.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tf-cifar10-example:latest


### Training(학습)

In [39]:
from sagemaker.estimator import Estimator

hyperparameters = {'train-steps': 100}

instance_type = 'ml.m4.xlarge'

estimator = Estimator(role=role,
                      instance_count=1,
                      instance_type=instance_type,
                      image_uri=ecr_image,
                      hyperparameters=hyperparameters)

estimator.fit(data_location)

2021-03-20 08:47:00 Starting - Starting the training job...
2021-03-20 08:47:02 Starting - Launching requested ML instancesProfilerReport-1616230020: InProgress
......
2021-03-20 08:48:30 Starting - Preparing the instances for training......
2021-03-20 08:49:30 Downloading - Downloading input data...
2021-03-20 08:49:50 Training - Downloading the training image......
2021-03-20 08:50:50 Training - Training image download completed. Training in progress....
2021-03-20 08:51:30 Uploading - Uploading generated training model
2021-03-20 08:51:30 Completed - Training job completed
ProfilerReport-1616230020: NoIssuesFound
Training complete.
Training seconds: 121
Billable seconds: 121


### Deployment(추론)

In [40]:
%%time
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer
predictor = estimator.deploy(1, instance_type, serializer=JSONSerializer(), deserializer=JSONDeserializer())

---------------!CPU times: user 290 ms, sys: 0 ns, total: 290 ms
Wall time: 7min 32s


In [41]:
import cv2
import numpy
image = cv2.imread("data/cat.png", 1)

# resize, as our model is expecting images in 32x32.
image = cv2.resize(image, (32, 32))
data = {'instances': numpy.asarray(image).astype(float).tolist()}

# For more information on the predictor class.
# https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/predictor.py
predictor.predict(data)

{'predictions': [{'probabilities': [0.0128377527,
    0.105647139,
    0.0673772,
    0.124453992,
    0.138409868,
    0.429724306,
    0.0602907762,
    0.0282724891,
    0.0190955419,
    0.0138909211],
   'classes': 5}]}

## 리소스 삭제(옵션)

엔드포인트 사용이 완료되면 엔드포인트를 삭제합니다.

생성한 모든 학습작업, 모델, 엔드포인트 는 SageMaker에 기록되었으며 AWS 콘솔을 통해 확인할 수 있습니다.


In [42]:
predictor.delete_endpoint()

# Reference

- [BYOC 가이드](https://github.com/gonsoomoon-ml/churn-prediction-workshop2/blob/master/BYOC/README.md)
- [How Amazon SageMaker interacts with your Docker container for training](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html)
- [How Amazon SageMaker interacts with your Docker container for inference](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html)
- [CIFAR-10 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html)
- [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk)
- [Dockerfile](https://docs.docker.com/engine/reference/builder/)
- [scikit-bring-your-own](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb)
